### 0. Инициализация

In [ ]:
# Загружаем библиотеки необходимые для выполнения кода ноутбука.

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import logging

from utils.shortcuts import print_deep_mem_usage
from utils.s3 import print_bucket_contents, upload_file_to_s3

In [ ]:
# Настройка форматов
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format',  '{:,.2f}'.format)

# Отключаем INFO сообщения от matplotlib
logging.getLogger('matplotlib').setLevel(logging.WARNING)

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (15,5)
plt.rcParams['axes.titlesize'] = 'large'
plt.rcParams['axes.labelsize'] =  'large'
plt.rcParams['xtick.labelsize'] =  'medium'
plt.rcParams['ytick.labelsize'] =  'medium'
plt.rcParams['legend.title_fontsize'] = 'large'
plt.rcParams['legend.fontsize'] = 'medium'
plt.rcParams['legend.framealpha'] = .9

### 1. Первый этап

#### 1.1 Загрузка первичных данных

In [ ]:
tracks = pd.read_parquet('data/tracks.parquet')
cat_names = pd.read_parquet('data/catalog_names.parquet')
interactions = pd.read_parquet('data/interactions.parquet')

#### 1.2 Обзор данных

##### 1.2.1 cat_names - справочник каталожных имен

In [ ]:
# Типы данных
cat_names.info(memory_usage='deep')

In [ ]:
# Визуальный осмотр данных
cat_names.head(3)

In [ ]:
# Диапазон значений id
cat_names['id'].describe()[['min', 'max', 'count']].astype('int')

In [ ]:
# Оптимизируем типы данных
cat_names = cat_names.reset_index(drop=True)
cat_names['id'] = cat_names['id'].astype('int32')
cat_names['type'] = cat_names['type'].astype('category')
cat_names['name'] = cat_names['name'].astype('category')

# Объем памяти после оптимизации типов
print_deep_mem_usage(cat_names)

In [ ]:
# Уникальные типы каталожных единиц
cat_names['type'].value_counts()

In [ ]:
# Проверка на пропуски -> отсутствуют
cat_names.isna().sum()

In [ ]:
# Наличие дубликатов по полю 'id' -> присутствуют
cat_names[['id']].duplicated().sum()

In [ ]:
# Наличие дубликатов по связке 'id', 'type' -> отсутсвуют
cat_names[['id', 'type']].duplicated().sum()

In [ ]:
# Наличие дубликатов по связке 'type', 'name' -> присутствуют, и много
cat_names[['type', 'name']].duplicated().sum()

In [ ]:
# Наличие дубликатов по связке 'type', 'name' для разных типов
cat_type_name_duplicated = cat_names[cat_names[['type', 'name']].duplicated()]

for type, total_count in cat_names['type'].value_counts().to_dict().items():
    duplicated_count = len(cat_type_name_duplicated.query('type == @type'))
    display(
        f'Type [{type}] : '
        f'duplicated names {duplicated_count} of {total_count} '
        f'or {duplicated_count / total_count * 100:.1f}%.'
    )

del cat_type_name_duplicated, type, duplicated_count, total_count

__NB! В каталоге около половины треков и трети альбомов имеют повторяющиеся (дублированные) имена.__   
Решение: оставляем как есть, корректировку (слияние) дубликатов в справочниках пока не проводим.

##### 1.2.2 tracks - данные о треках

In [ ]:
tracks.info(memory_usage='deep')

In [ ]:
# визуальный отсмотр
tracks.head(3)

In [ ]:
# Оптимизируем типы даныых
tracks = tracks.reset_index(drop=True)
tracks['track_id'] = tracks['track_id'].astype('int32')
tracks[['albums', 'artists', 'genres']] = (
    tracks[['albums', 'artists', 'genres']]
    .map(lambda x: tuple(x.astype('int32')))
)

# Объем в памяти после оптимизации
print_deep_mem_usage(tracks)

In [ ]:
# Исходное число треков
track_count_initial = len(tracks)

In [ ]:
# дубликаты по 'track_id' -> отсутствуют
tracks['track_id'].duplicated().sum()

In [ ]:
# Все ли track_id есть в справочнике -> да, все
set(tracks['track_id']) - set(cat_names.query('type == "track"')['id'])

In [ ]:
# все ли id альбомов есть в справочнике -> нет, не все: 
# встречаются пропуски (nan)
(
    set(tracks['albums'].explode().drop_duplicates()) 
    - set(cat_names.query('type == "album"')['id'])
)

In [ ]:
# визуальный осмотр битых идентификаторов альбомов
track_ids_with_broken_albums = (
    tracks
    .explode('albums')
    .query('albums.isna()')
    ['track_id']
    .drop_duplicates()
)
print(
    f'Tracks with broken "albums": {len(track_ids_with_broken_albums)} '
    f'or {len(track_ids_with_broken_albums) / track_count_initial :.2%} '
    'of total.\n\nSample:'
)
tracks.query('track_id in @track_ids_with_broken_albums').head(3)

__Проблема__: в данных о треках `tracks` есть 18 треков с отсутствующими (пустыми) значениями id альбомов.      
_Решение_: удаляем такие треки из справочника (каталога)

In [ ]:
# удаляем треки с битыми (пустыми) альбомами
tracks = (
    tracks
    .query('track_id not in @track_ids_with_broken_albums')
    .reset_index(drop=True)
)
del track_ids_with_broken_albums

In [ ]:
# все ли id артистов есть в справочнике -> нет, не все: 
# встречаются пропуски (nan)
(
    set(tracks['artists'].explode().drop_duplicates()) 
    - set(cat_names.query('type == "artist"')['id'])
)

In [ ]:
# визуальный осмотр битых идентификаторов артистов
track_ids_with_broken_artists = (
    tracks
    .explode('artists')
    .query('artists.isna()')
    ['track_id']
    .drop_duplicates()
)
print(
    f'Tracks with broken "artists": {len(track_ids_with_broken_artists)} '
    f'or {len(track_ids_with_broken_artists) / track_count_initial :.2%} '
    'of total.\n\nSample:'
)
tracks.query('track_id in @track_ids_with_broken_artists').head(3)

__Проблема__: в данных о треках `tracks` есть ~1.5% треков с отсутствующими (пустыми) значениями id артистов.      
_Решение_: удаляем такие треки из справочника (каталога)

In [ ]:
# удаляем треки с битыми (пустыми) идентификаторами артистов
tracks = (
    tracks
    .query('track_id not in @track_ids_with_broken_artists')
    .reset_index(drop=True)
)

del track_ids_with_broken_artists

In [ ]:
# все ли id жанров есть в справочнике -> нет, не все: 
# встречаются как пропуски (nan) так и безимянные жанры
genre_ids_missing = (
    set(tracks['genres'].explode().drop_duplicates()) 
    - set(cat_names.query('type == "genre"')['id'])
)
print(*genre_ids_missing)

In [ ]:
# визуальный осмотр битых идентификаторов жанров
track_ids_with_broken_genres = (
    tracks
    .explode('genres')
    .query('genres in @genre_ids_missing') 
    ['track_id']
    .drop_duplicates()
)
print(
    f'Tracks with broken "genres": {len(track_ids_with_broken_genres)} '
    f'or {len(track_ids_with_broken_genres) / track_count_initial :.2%} '
    'of total.\n\nSample:'
)
tracks.query('track_id in @track_ids_with_broken_genres').head(3)

__Проблема__: в данных о треках `tracks` есть ~5% треков с неизвестными или отсутствующими (пустыми) значениями id жанров.      
_Решение_: удаляем такие треки из справочника (каталога)

In [ ]:
# удаляем треки с неизвестными иои пустыми идентификаторами жанров
tracks = (
    tracks
    .query('track_id not in @track_ids_with_broken_genres')
    .reset_index(drop=True)
)
del genre_ids_missing, track_ids_with_broken_genres

In [ ]:
# сколько треков осталось после чистки данных
print(
    f'Cleaned tracks: {len(tracks)} '
    f'or {len(tracks) / track_count_initial:.2%} of inital count.'
)

In [ ]:
# Приборка
del track_count_initial

##### 1.2.3 interactions - взаимодействия пользователей с треками

In [ ]:
interactions.info(memory_usage='deep')

In [ ]:
# визуальный осмотр данных
interactions.head(3)

In [ ]:
# Посмотрим на фактическую дискретность временных меток
interactions['started_at'].drop_duplicates().sort_values().diff().min()

In [ ]:
# Оптимизация типов 
interactions = interactions.reset_index(drop=True)
interactions['started_at'] = interactions['started_at'].astype('category')

# Размер в памяти после оптимизации
print_deep_mem_usage(interactions)

In [ ]:
# исходное число записей о взаимодействиях
interactions_count_initial = len(interactions)

In [ ]:
# Оценим кол-во взаимодействий с треками отсутствующими в справочнике tracks

# ids известных треков
known_track_ids = set(tracks['track_id'])

# взаимодействия с неизвестными треками
interactions_with_unknown_tracks = (
    interactions.query("track_id not in @known_track_ids")
)
print(
    'Interactions with unknown tracks: '
    f'{len(interactions_with_unknown_tracks) / interactions_count_initial:.2%} '
    'of total.'
)
del interactions_with_unknown_tracks

Около 3% взаимодействий приходятся на треки, отсутствующие в каталоге (справочнике) `tracks`.       
_Решение_ Удаляем такие прослушивания из `interactions`

In [ ]:
# Удаляем данные о прослушивании 'неизвестных' (или 'битых') треков
interactions = (
    interactions
    .query('track_id in @known_track_ids')
    .reset_index(drop=True)
)
del known_track_ids

In [ ]:
# сколько взаимодействий осталось после чистки данных
print(
    f'Cleaned interactions: {len(interactions)} '
    f'or {len(interactions) / interactions_count_initial:.2%} of inital count.'
)

In [ ]:
# Приборка
del interactions_count_initial

#### 1.3 Выводы

Проблемы с данными в исходных датасетах:
- в каталоге `cat_names` около половины(!) треков и трети альбомов имеют повторяющиеся (дублированные) имена,    
    - _Решение_: оставляем как есть, корректировку (слияние) дубликатов в справочниках пока не проводим.
- в данных о треках `tracks` есть 18 треков с отсутствующими (пустыми) значениями id альбомов,
    - _Решение_: удаляем такие треки из справочника (каталога)
- в данных о треках `tracks` есть ~1.5% треков с отсутствующими (пустыми) значениями id артистов.      
    - _Решение_: удаляем такие треки из справочника (каталога)
- в данных о треках `tracks` есть ~5% треков с неизвестными или отсутствующими (пустыми) значениями id жанров.      
    - _Решение_: удаляем такие треки из справочника (каталога)
- Около 3% взаимодействий приходятся на треки, отсутствующие в 'почищенном' каталоге (справочнике) `tracks`.       
    - _Решение:_ Удаляем такие прослушивания из `interactions`
- очевидных проблем с идентификаторами (ids) не обнаружено

### 2. Второй этап

#### 2.1 Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

In [ ]:
# Создадим объеккт items
items = (
    tracks.copy().rename(columns={'track_id': 'item_id'}).reset_index(drop=True)
)

In [ ]:
# Добавим к items строковое имя трека из каталога
items = items.join(
    cat_names.query('type == "track"').set_index('id')['name'],
    on='item_id'
).rename(columns={'name': 'item_name'})

# Визуальная проверка
items.head(3)

In [ ]:
# Добавляем к items имена альбомов, артистов и жанров из каталога

# Создаем словари типа {id: строковое_имя}
albums_names = (cat_names.query('type == "album"').set_index('id')['name']
               .to_dict())
artists_names = (cat_names.query('type == "artist"').set_index('id')['name']
                 .to_dict())
genres_names = (cat_names.query('type == "genre"').set_index('id')['name']
               .to_dict())

# Функции преобрабования кортежа с ids (int) в кортеж с именами (строками)
get_albums_names = lambda x: tuple(map(albums_names.__getitem__, x))
get_artists_names = lambda x: tuple(map(artists_names.__getitem__, x))
get_genres_names = lambda x: tuple(map(genres_names.__getitem__, x))

items['albums_names'] = items['albums'].map(get_albums_names)
items['artists_names'] = items['artists'].map(get_artists_names)
items['genres_names'] = items['genres'].map(get_genres_names)

del (albums_names, artists_names, genres_names, get_albums_names,
     get_artists_names, get_genres_names)

# Визуальна проверка
items.head(3)

In [ ]:
# Проверим, что items не раздуло в пямяти после присоеденинения строковых имен
print_deep_mem_usage(items, columns_info=False)

In [ ]:
# Создадим объект events
events = interactions.copy().rename(columns={'track_id': 'item_id'})

In [ ]:
# Приберемся - cat_names, interactions, tracks больше не нужны
del cat_names, interactions, tracks

#### 2.2 EDA

In [ ]:
events['user_id'].nunique()

Общее кол-во уникальных пользователей в выборке около 1.3 млн.

In [ ]:
# Кол-во прослушанных треков по пользователям
track_count_by_user = (
    events.groupby('user_id').agg(tracks_listened=('item_id', 'count'))
)

# Статистики по кол-ву прослушаных треков
track_count_by_user.describe().astype('int')

In [ ]:
ax = sns.histplot(track_count_by_user, stat='percent', binwidth=5)
ax.set_xlim(0, 300)
ax.axvline(track_count_by_user['tracks_listened'].median(), color='blue',
           linestyle='-', label='Median')
ax.axvline(track_count_by_user['tracks_listened'].mean(), color='red',
           linestyle='-', label='Mean')
ax.legend()
ax.set_title('Распределение числа треков прослушанных '
             'одним пользователем')
ax.plot()
del ax, track_count_by_user

Медианное кол-ва прослушанных пользователем треков 54.      
Из-за наличия достаточного кол-ва меломанов с большим числом прослушанных треков (>1000) среднее существенно выше и составляет 156.

Расчитаем индекс популярности треков:
- для каждого трека считаем кол-во прослушавших его пользователей
- нормируем от 0 до 1 (делим на кол-во пользователей, прослушавших самый популярный трек)

In [ ]:
# Расчитаем индекс популярности треков:
item_popularity = (
    events
    .groupby('item_id')
    .agg(popularity=('user_id', 'count'))
    .reset_index()
)
item_popularity['popularity'] /= item_popularity['popularity'].max()
item_popularity['popularity'] = item_popularity['popularity'].astype('float32')

# Добавим индекс популярности в таблицу items
items = items.merge(item_popularity, on='item_id', how='left')
items['popularity'] = items['popularity'].fillna(0)

del item_popularity

In [ ]:
# Наиболее популярные треки
(
    items[['item_name', 'artists_names', 'genres_names', 'albums_names',
           'popularity']]
    .sort_values(by='popularity', ascending=False)
    .head(10)
)

Аналогичным образом расчитаем индекс популярности жанров:
- для каждого жанра считаем общее кол-во прослушиваний треков данного жанра,
- нормируем от 0 до 1 (делим на кол-во прослушиваний для самого популярного жанра)

In [ ]:
# Для удобства (вос)создадим справочную таблицу по жанрам
genres = (
    items[['genres', 'genres_names']]
    .explode(['genres', 'genres_names'])
    .drop_duplicates()
    .rename(columns={'genres': 'genre_id', 'genres_names': 'genre_name'})
    .sort_values(by='genre_id')
    .reset_index(drop=True)
)

# Визуальная проверка
genres.head(3)

In [ ]:
# Сначала (снова) посчитаем кол-во прослушиваний для каждого трека
events_count = (
    events
    .groupby('item_id')
    .agg(events_count=('user_id', 'count'))
    .reset_index()
)

# После агрегации по трекам добавим информацию по жанрам
# и распакуем кортежи с идентифиакаторами жанров
events_count = (
    events_count
    .merge(items[['item_id', 'genres']], on='item_id', how='left')
    .explode('genres')
    .rename(columns={'genres': 'genre_id'})
)

# визуальная проверка
events_count.head(3)

In [ ]:
# И наконец аггрегируем по жанрам для расчета суммарного кол-ва прослушанных
# треков по каждому жанру 
genre_popularity = (
    events_count
    .groupby('genre_id')
    .agg(popularity=('events_count', 'sum'))
)
# Нормируем
genre_popularity['popularity'] /= genre_popularity['popularity'].max()
genre_popularity['popularity'] = (genre_popularity['popularity']
                                  .astype('float32'))

# Добавляем индекс популярности к справочной таблице по жанрам
genres = genres.merge(genre_popularity, on='genre_id', how='left')
genres['popularity'] = genres['popularity'].fillna(0)

# Прибираемся
del events_count, genre_popularity

# Наиболее популярные жанры
genres.sort_values(by='popularity', ascending=False).head(10)

Неожиданный факт: наиболее популярным является жанр с именем `pop`[ular].      


Для выделения треков, которые никто не прослушал можно воспользоваться уже рассчитанным индексом популярности трека, для непрослушанных треков индекс популярности равен нулю.

In [ ]:
# Количество никем не прослушанных треков
(items['popularity'] == 0).sum()

In [ ]:
# Альтернативный способ подсчета непрослушанных треков
set(items['item_id']) - set(events['item_id'].drop_duplicates())

Кто-то уже позаботился об очистке исходных данных - непрослушанные треки в справочнике (катаологе) треков отсутствуют.

#### 2.3 Сохранение данных

In [ ]:
# Сохраним данные локально

# NB: признак popularuty для items, genres сохранять не будем,
# т.к. мы его считали по всей выборке событий, включая будущую тестовую выборку
items.drop(columns='popularity').to_parquet('data/items.parquet')
events.to_parquet('data/events.parquet')
genres.drop(columns='popularity').to_parquet('data/genres.parquet')

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

In [ ]:
# Заливаем файлы
upload_file_to_s3('data/items.parquet', 'recsys/data/items.parquet')
upload_file_to_s3('data/events.parquet', 'recsys/data/events.parquet')

# Визуальная проверка содержимого бакета 
print_bucket_contents(key_pattern='recsys')

#### 2.4 Очистка памяти - перезагружаем ядро

Можно было бы ограничится удалением переменных, но перезагрузка ядра перед 'тяжелыми' расчетами более надежный вариант.

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

### 3. Третий этап

#### 3.1 Повторная инициализация после перезагрузки ядра

In [1]:
# Повторно загружаем библиотеки необходимые для выполнения кода ноутбука.
import pickle

import numpy as np
import pandas as pd
import scipy
import tqdm as notebook_tqdm
from implicit.als import AlternatingLeastSquares
from sklearn.preprocessing import LabelEncoder

from utils.s3 import print_bucket_contents, upload_file_to_s3
from utils.shortcuts import (print_csr_mem_usage, print_deep_mem_usage,
                             print_total_mem_usage)

/home/mle-user/mle_projects/mle-recsys-project/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from threadpoolctl import threadpool_limits

# Следуем рекомендациям модуля implicit для оптимизации ALS
threadpool_limits(1, "blas");

In [3]:
# Настройка форматов pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format',  '{:,.2f}'.format)

In [4]:
N_RECS_USER = 50    # число рекомендаций модели для одного пользователя

#### 3.2 Загрузка данных

In [5]:
# Загружаем данные в пямять
items = pd.read_parquet('data/items.parquet')
events = pd.read_parquet('data/events.parquet')

In [6]:
# Объем данных в пямяти после загрузки
print('Items\t: ', end=''); print_deep_mem_usage(items, columns_info=False)
print('Events\t: ', end=''); print_deep_mem_usage(events, columns_info=False)

Items	: Total memory usage 757.4 MB
Events	: Total memory usage 3685.9 MB


In [7]:
# Для 'списочных' элементов В items пережмём ndarrays в кортежи 
# для экономии пямяти
list_like_columns = ['albums', 'artists', 'genres', 'albums_names', 
                     'artists_names', 'genres_names']

items[list_like_columns] = items[list_like_columns].map(lambda x: tuple(x))

del list_like_columns

# Визуальная проверка результата
items.head(3)

,item_id,albums,artists,genres,item_name,albums_names,artists_names,genres_names
0,26,"(3, 2490753)","(16,)","(11, 21)",Complimentary Me,"(Taller Children, Taller Children)","(Elizabeth & the Catapult,)","(pop, folk)"
1,38,"(3, 2490753)","(16,)","(11, 21)",Momma's Boy,"(Taller Children, Taller Children)","(Elizabeth & the Catapult,)","(pop, folk)"
2,135,"(12, 214, 2490809)","(84,)","(11,)",Atticus,"(Wild Young Hearts, Wild Young Hearts, Wild Yo...","(Noisettes,)","(pop,)"


In [8]:
# В events изменим тип для started_at на категориальный 
# NB: временная дискретность - дневная
events['started_at'] = events['started_at'].astype('category').cat.as_ordered()

In [9]:
# Объем в пямяти после оптимизации типов
print('Items\t: ', end=''); print_deep_mem_usage(items, columns_info=False)
print('Events\t: ', end=''); print_deep_mem_usage(events, columns_info=False)

Items	: Total memory usage 459.1 MB
Events	: Total memory usage 2457.3 MB


#### 3.3 Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.        
Дата разделения __16 декабря 2022__ (первый день для тестовых данных)

In [10]:
events_train_idx = (events['started_at'] < pd.to_datetime('2022-12-16'))

events_train = events[events_train_idx].reset_index(drop=True)
events_test = events[~events_train_idx].reset_index(drop=True)

del events, events_train_idx 

In [11]:
# Посмотрим на разницу в множествах пользователей 
# в обущающей и тестовой выборке 
users_train = set(events_train['user_id'].drop_duplicates())
users_test = set(events_test['user_id'].drop_duplicates())

# Холодные пользователи - отсутствующие в обучающей выборке
cold_users = users_test - users_train

print(f'Пользователй в обучающей выборке : {len(users_train):>10}')
print(f'Пользователй в тестовой выборке  : {len(users_test):>10}')
print(f'  в т.ч.холодные пользователи    : {len(cold_users):>10}')

Пользователй в обучающей выборке :    1341269
Пользователй в тестовой выборке  :     778789
  в т.ч.холодные пользователи    :      30773


In [12]:
del users_test, users_train, cold_users
print_total_mem_usage()

Process memory usage: 5.48 GB


#### 3.4 Топ популярных

Расчитаем индекс (score) популярности как общее кол-во прослушиваний трека различными пользователями.       
- Отнормируем от 0 до 1, разделив на максимальное число прослушиваний одного трека.             
- Используем только обучающую выборку, берем N_RECS_USER наиболее популярных.        

___NB:___ Особенность наших данных - данные о преслушивании трека конкретным пользователем включены в events только один раз, другими словами, комбинация (user_id, item_id) в пределах таблицы events уникальна.

In [13]:
top_popular = (
    events_train
    .groupby('item_id')
    .agg(score=('user_id', 'count'))
    .reset_index()
    .sort_values(by='score', ascending=False)
    [:N_RECS_USER]
    .reset_index(drop=True)
)
top_popular['score'] /= top_popular['score'].max()
top_popular['score'] = top_popular['score'].astype('float32')

# визуальная проверка 
top_popular.head(3)

,item_id,score
0,53404,1.00
1,33311009,0.92
2,178529,0.92


In [14]:
# Сохраняем локально
top_popular.to_parquet('data/top_popular.parquet')

In [15]:
# Для пользвателей из тестовой выборки подготовим таблицу с
# рекомендациями вида (user_id, item_id, score)  
top_popular_recs = (
    events_test[['user_id']]
    .drop_duplicates()
    .reset_index(drop=True)
    .merge(top_popular, how='cross')
    .reset_index(drop=True)
)
top_popular_recs.head(3)

,user_id,item_id,score
0,3,53404,1.00
1,3,33311009,0.92
2,3,178529,0.92


Подготовим функции для расчета различных метрик полученных рекомпндаций:
- precision@K1, recall@K1 - точность и полнота для среза топ-K1 рекомендаций
- coverage@K2 - покрытие по объектам для cреза топ-K2 рекомендаций
- novelty@K3 - новизна по объектам для среза топ-K3 рекомендаций

In [16]:
def get_precision_recall_metrics(
    recs: pd.DataFrame,     
    events_test: pd.DataFrame,  
    K_prc_rec: int = 5
):
    """Calculate precision@K, recall@K metrics.

    Parameters
    ----------
    - recs: reccomendations table containing `user_id, item_id, score` columns
    - events_test: events table containing `user_id, item_id`
    - K_prc_rec: at_K value for precision and recall 
    """

    # Расчет будем вести по пользователям, удовлетворяющим двум условиям:
    # 1) для них были даны рекомендации
    # 2) они взаимодействовали с объектами в тестовый период 
    effective_users = (
        set(recs['user_id'].drop_duplicates()) 
        & set(events_test['user_id'].drop_duplicates())
    )
    events_test = (
        events_test
        .query('user_id in @effective_users')
        .reset_index(drop=True)
        .copy()
    )
    recs = (
        recs
        .query('user_id in @effective_users')
        # Оставляем только top-K рекомендаций для каждого пользователя
        .sort_values(by=['user_id', 'score'], ascending=[True, False])
        .groupby('user_id')
        .head(K_prc_rec)
        .reset_index(drop=True)
        .copy()
    )
    # Философский вопрос состоит в том, нужно ли удалять из тестовой выборки
    # взаимодействия с объектами, отсутствующими в обучающей выборке?
    # Примем решение, что мы так делать не будем.

    # Помечаем события в тестовой выборке как 'ground truth'
    events_test['gt'] = True

    # Сливаем события тестовой выборки с рекомендациями
    events_recs = (
        events_test[['user_id', 'item_id', 'gt']]
        .merge(
            recs[['user_id', 'item_id', 'score']],
            on=['user_id', 'item_id'],
            how='outer'
        )
    )
    # Для нерелевантных рекомендации 'gt': NaN заменяем на False 
    events_recs['gt'] = events_recs['gt'].fillna(False)

    # Размечаем рекомендованные объекты (имеющие score) как 'rec': True
    events_recs['rec'] = events_recs['score'].notna()

    # Размечаем true positive, false positive и false negative
    events_recs['tp'] = events_recs['gt'] & events_recs['rec']
    events_recs['fp'] = ~events_recs['gt'] & events_recs['rec']
    events_recs['fn'] = events_recs['gt'] & ~events_recs['rec']

    grouped = events_recs.groupby('user_id')

    precision = (
        grouped['tp'].sum() / (grouped['tp'].sum() + grouped['fp'].sum())
    )
    precision = precision.fillna(0).mean()

    recall = (
        grouped['tp'].sum() / (grouped['tp'].sum() + grouped['fn'].sum())
    )
    recall = recall.fillna(0).mean()

    return (precision, recall)

In [17]:
def get_item_coverage_metric(
    recs: pd.DataFrame,     
    items: pd.DataFrame,    
    K_coverage: int = N_RECS_USER,
):
    """
    Calculate and return coverage@K metrics.

    Parameters
    ----------
    - recs: reccomendations table containing `user_id, item_id, score` columns
    - items: all items table containing `item_id` column
    - K_coverage: at_K value for coverage
    """

    # Берем срез топ-К
    top_k_recs = (
        recs
        .sort_values(by=['user_id', 'score'], ascending=[True, False])
        .groupby('user_id')
        .head(K_coverage)
    )
    
    # кол-во уникальных объектов в К-срезах к общему кол-ву объектов
    return top_k_recs['item_id'].nunique() / len(items)

In [18]:
def get_novelty_metric(
    recs: pd.DataFrame,   
    events_train: pd.DataFrame,
    K_novelty: int = 5
):
    """
    Calculate and return novelty@K metric.

    Parameters
    ----------
    - recs: reccomendations table containing `user_id, item_id, score` columns
    - events_trian: events table containing `user_id, item_id`
    - K_novelty: at_K value for novelty
    """

    # Отфильтруем рекомендации, оставив только топ-К
    top_k_recs = (
        recs
        .sort_values(by=['user_id', 'score'], ascending=[True, False])
        .groupby('user_id')
        .head(K_novelty)
        .copy()
    )
    # Добавим к обучающим событиям колонку 'new' : False 
    # - признак прослушивания трека конкретным пользователем
    events_train['new'] = False
    
    # Добавим признак 'new' к таблице рекомендаций
    top_k_recs = top_k_recs.merge(
        events_train[['user_id', 'item_id', 'new']],
        on=['user_id', 'item_id'],
        how='left'
    )
    top_k_recs['new'] = top_k_recs['new'].fillna(True).astype('bool')

    # Для топ-К рекомендаций расчитаем novelty по пользователям и усредним
    return top_k_recs.groupby('user_id')['new'].mean().mean()    

In [19]:
def get_recs_metrics(
    recs: pd.DataFrame,     
    items: pd.DataFrame,    
    events_test: pd.DataFrame,
    events_train: pd.DataFrame,
    K_prc_rec: int = 5,
    K_coverage: int = N_RECS_USER,
    K_novelty: int = 5
):
    """
    Calculate and return precision@K, recall@K, coverage@K, novelty@K metrics.

    Parameters
    ----------
    - recs: reccomendations table containing `user_id, item_id, score` columns
    - items: all items table containing `item_id` column
    - events_test: events table containing `user_id, item_id`
    - events_trian: events table containing `user_id, item_id`
    - K_prc_rec: at_K value for precision and recall 
    - K_coverage: at_K value for coverage
    - K_novelty: at_K value for novelty
    """
    precision_at_K, recall_at_K = (
        get_precision_recall_metrics(recs, events_test, K_prc_rec)
    )
    coverage_at_K = get_item_coverage_metric(recs, items, K_coverage)
    novelty_at_K = get_novelty_metric(recs, events_train, K_novelty)
    
    return {
        f'Precision@{K_prc_rec}, %': precision_at_K * 100,
        f'Recall@{K_prc_rec}, %': recall_at_K * 100,
        f'Coverage@{K_coverage}, %': coverage_at_K * 100,
        f'Novelty@{K_novelty}, %': novelty_at_K * 100,
    }


In [20]:
metrics = pd.DataFrame(
    get_recs_metrics(top_popular_recs, items, events_test, events_train),
    index=['Top popular'])
metrics

,"Precision@5, %","Recall@5, %","Coverage@50, %","Novelty@5, %"
Top popular,0.36,0.13,0.01,92.62


Для рекомендаций топ популярных метрики ожидаемо не ахти, но неожиданно высокая novelty 😊

In [21]:
print_total_mem_usage()

Process memory usage: 6.40 GB


#### 3.5 Персональные ALS

__NB:__ Персональные рекомендации считаем только для пользователй и объектов  [__!__], входящих в обучающую выборку events_train (< 16 декабя 2022 года). 

In [22]:
# Создаем энкодеры для перекодирования идентификаторов user_id, item_id 
# в натруральный ряд  {0, 1, ...} для построения матрицы взаимодействий
# NB: пользователи и объекты - только из обучающей выборки [!]

user_encoder = LabelEncoder().fit(
    events_train['user_id'].drop_duplicates().sort_values()
)
item_encoder = LabelEncoder().fit(
    events_train['item_id'].drop_duplicates().sort_values()
)

In [23]:
# создаём sparse-матрицу (user x item) 
user_item_matrix = scipy.sparse.csr_matrix(
    # кортеж вида (data, (row_index, col_index))
    (
        np.ones(len(events_train), dtype=np.int8),  # взаимодействие -> единица
        (
            user_encoder.transform(events_train['user_id']),
            item_encoder.transform(events_train['item_id'])
        )
    )
)

In [27]:
# Для экономии времени - загружаем обученную модель
with open('models/als_model.pkl', 'rb') as f:
    als_model = pickle.load(f)
    
# Проврерочный расчет
als_model.recommend(
    1000, 
    user_item_matrix[1000], 
    filter_already_liked_items=True,
    N=10
)

(array([  4522,  27441,  21898,  10462,   5484,  52000,   2015, 208618,
          2903,  39098], dtype=int32),
 array([0.01942061, 0.018442  , 0.01827359, 0.01789448, 0.01765682,
        0.01726999, 0.01719784, 0.01719521, 0.01658652, 0.0159805 ],
       dtype=float32))

In [37]:
# Время выполения ~45 минут
# Получаем рекомендации для всех пользователй из events_train
als_user_ids, als_scores = als_model.recommend(
    range(len(user_encoder.classes_)),  # все закодированные user_id
    user_item_matrix, 
    filter_already_liked_items=True,
    N=N_RECS_USER
)

In [48]:
# в предыдущей ячейке ошиблись с именем переменной, в als_user_ids положили 
# идентификаторы рекомендуемых объектов, исправляем:
als_item_ids = als_user_ids

In [71]:
# Перепаковываем рекомендации для всех пользователй в таблицу формата
# (user_id, item_id, score)
personal_als = pd.DataFrame({
    'user_id': range(len(user_encoder.classes_)),
    'item_id': als_item_ids.tolist(),
    'score': als_scores.tolist()
})
personal_als = personal_als.explode(['item_id', 'score'])
personal_als['user_id'] = personal_als['user_id'].astype('int32')
personal_als['item_id'] = personal_als['item_id '].astype('int32')
personal_als['score'] = personal_als['score '].astype('float32')

: 

In [57]:
personal_als.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1341269 entries, 0 to 1341268
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   user_id  1341269 non-null  int64 
 1   item_id  1341269 non-null  object
 2   score    1341269 non-null  object
dtypes: int64(1), object(2)
memory usage: 30.7+ MB


In [59]:
print_total_mem_usage()

Process memory usage: 21.11 GB


#### 3.6 Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

#### 3.7 Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

#### 3.8 Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

#### 3.9 Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

#### 3.10 Выводы, метрики

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.